# Mounting Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

!ls drive/My\ Drive

# # Uncomment the following for an example of saving a Keras model to your Google Drive using callbacks
# # WARNING: THIS WILL EDIT YOUR GOOGLE DRIVE.  USE AT YOUR OWN CAUTION

# # Imports
# import numpy as np
# from keras.layers import Dense
# from keras.models import Sequential
# from keras.callbacks import ModelCheckpoint

# # Fake data
# x = np.random.random((32, 4))
# y = np.random.random(32)

# # Model
# model = Sequential([
#         Dense(1, input_shape=(4,))
# ])

# model.compile(optimizer='sgd', loss='mse')

# # This is what saves the model to your drive
# model_checkpointer = ModelCheckpoint('/content/drive/My Drive/model.h5')

# # Training
# model.fit(x, y, epochs=2, callbacks=[model_checkpointer])

# Uploading and Downloading Files
Upload files with:

```python
from google.colab import files
files.upload()
```

And download files with:
```python
from google.colab import files
files.download('path/to/your/file')
```


# Form-Fields

In [0]:
#@title Example form fields
#@markdown Forms support many types of fields.

no_type_checking =   ''#@param
string_type = 'example'  #@param {type: "string"}
slider_value = 160  #@param {type: "slider", min: 100, max: 200}
number = 102  #@param {type: "number"}
date = '2010-11-05'  #@param {type: "date"}
pick_me = "monday"  #@param ['monday', 'tuesday', 'wednesday', 'thursday']
select_or_input = "apples" #@param ["apples", "bananas", "oranges"] {allow-input: true}
#@markdown ---

In [0]:
import numpy as np
from keras.layers import Dense, Input
from keras.models import Model
from keras.callbacks import ModelCheckpoint

# Fake data
x_data = np.random.random((32, 4))
y_data = np.random.random((32, 1))

# Hyperparameters
EPOCHS = 10 #@param {type: "number"}
BATCH_SIZE = 16 #@param {type: "slider", min: 2, max: 32, step: 2}
NUM_DENSE_LAYERS = 4 #@param {type: "slider", min: 1, max: 6}
UNITS_PER_DENSE_LAYER = 16 #@param {type: "number"}
ACTIVATION = 'elu' #@param ['relu', 'tanh', 'elu', 'sigmoid', 'linear']
OPTIMIZER = 'sgd' #@param ['sgd', 'RMSprop', 'adam', 'adagrad', 'adadelta', 'adamax', 'nadam']
LOSS = 'mse' #@param ['mse', 'mae', 'squared_hinge', 'hinge', 'cosine_proximity']

# Model
input_layer = Input(shape=(4,))
x = Dense(UNITS_PER_DENSE_LAYER, activation=ACTIVATION)(input_layer)

for _ in range(NUM_DENSE_LAYERS-1):
    x = Dense(UNITS_PER_DENSE_LAYER, activation=ACTIVATION)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(input_layer, x)
model.compile(optimizer=OPTIMIZER, loss=LOSS)

model.summary()

# Training
print(x_data.shape, model.input_shape, y_data.shape, model.output_shape)
model.fit(x_data, y_data, epochs=EPOCHS, batch_size=BATCH_SIZE)

# Running JavaScipt and HTML

In [0]:
import IPython

js_code = \
'''
let message = "Hello world!";
document.querySelector("#output-area").appendChild(document.createTextNode(message));
'''

display(IPython.display.Javascript(js_code))

In [0]:
%%javascript

let message = "Javascript in Google Colab? Awesome!"
console.log(message) // check your console with F12 or ctrl/cmnd+shift+i and swapping to the `Console` tab
document.querySelector("#output-area").appendChild(document.createTextNode(message));

In [0]:
html_code = \
'''
<p>Now this is <i>really</i> awesome!</p>
<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/0/0a/Python.svg/768px-Python.svg.png'></img>
'''

display(IPython.display.HTML(html_code))

# Using the Webcam

## Basic Usage
The example given by the Colab Team

In [0]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [0]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

## Save Image to NumPy (NP) Array

In [0]:
import tempfile
import cv2

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])

  # Create temporary file for holding image data
  img_file = tempfile.NamedTemporaryFile()
  img_file.write(binary)

  # Read temp. file and convert it to a Numpy array
  img = cv2.imread(img_file.name)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  # Close file
  img_file.close()

  return img

In [0]:
import matplotlib.pyplot as plt

img = take_photo()

plt.imshow(img)
plt.show()

## Instantly take a photo and save it to a NumPy Array

In [0]:
import tempfile
import cv2

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Waiting any shorter than this seems to make the image really red
      // You may want to remove this depending on your webcam
      await new Promise(r => setTimeout(r, 175))

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])

  # Create temporary file for holding image data
  img_file = tempfile.NamedTemporaryFile()
  img_file.write(binary)

  # Read temp. file and convert it to a Numpy array
  img = cv2.imread(img_file.name)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  # Close file
  img_file.close()

  return img

In [0]:
import matplotlib.pyplot as plt

img = take_photo()

plt.imshow(img)
plt.show()